In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
import math
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")


gpu


In [20]:
train_dataset = dsets.FashionMNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.FashionMNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())
print(train_dataset.train_data.size())
print(test_dataset.test_data.size())

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [21]:
classes = ('T-Shirt','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot')
def imshow(img):
    npimg = img.numpy() #convert the tensor to numpy for displaying the image
    #for displaying the image, shape of the image should be height * width * channels 
    plt.imshow(np.transpose(npimg, (1, 2, 0))) 
    plt.show()

In [23]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [24]:
class CNNModel(nn.Module):
    def __init__(self, width):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        # Max pool 1
        # self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        kernel = 3
        kernel_max2 = 2
        stride1,stride2 = 1,1
        width_cnn1 = math.ceil((width-kernel+2)/stride1 + 1 )
        print("width_cnn1: ", width_cnn1)
        width_cnn2 = math.ceil((width_cnn1-kernel+2)/stride2 + 1 )
        print("width_cnn2: ", width_cnn2)
        width_max2 = math.ceil((width_cnn2-kernel_max2)/kernel_max2 + 1)
        print("width_max2: ", width_max2)
        self.fc1 = nn.Linear(32 * width_max2 * width_max2, 128)
        self.relu3 = nn.ReLU() 
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        # Convolution 1
        # print("x size: ", x.size())
        out = self.cnn1(x)
        # print("cnn1 size: ", out.size())
        out = self.relu1(out)
        # print("relu1 size: ", out.size())
        # Max pool 1
        # print("bef: ", out.size())
        # out = self.maxpool1(out)
        # print("aft: ", out.size())
        # Convolution 2 
        out = self.cnn2(out)
        # print("cnn2 size: ", out.size())
        out = self.relu2(out)
        # print("rel2 size: ", out.size())
        # Max pool 2 
        out = self.maxpool2(out)
        # print("max2 size: ", out.size())
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        # print("view size: ", out.size())
        out = self.fc1(out)
        out = self.relu3(out)

        #dropout
        out = self.dropout(out)

        #linear func
        out = self.fc2(out)
        return out

In [25]:
model = CNNModel(28)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

width_cnn1:  28
width_cnn2:  28
width_max2:  14


In [26]:
print(model.parameters())

print(len(list(model.parameters())))

# Convolution 1: 16 Kernels
print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Convolution 2: 32 Kernels with depth = 16
print(list(model.parameters())[2].size())

# Convolution 2 Bias: 32 Kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer Bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7f7a1034c518>
8
torch.Size([16, 1, 3, 3])
torch.Size([16])
torch.Size([32, 16, 3, 3])
torch.Size([32])
torch.Size([128, 6272])
torch.Size([128])


In [27]:
iter = 0
for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_().to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        # Forward pass to get output/logits
        outputs = model(images)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_().to(device)
                labels = labels.to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct // total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.6801870465278625. Accuracy: 72
Iteration: 1000. Loss: 0.44436731934547424. Accuracy: 77
Iteration: 1500. Loss: 0.5709998607635498. Accuracy: 81
Iteration: 2000. Loss: 0.5649828910827637. Accuracy: 81
Iteration: 2500. Loss: 0.42584770917892456. Accuracy: 82
Iteration: 3000. Loss: 0.40394797921180725. Accuracy: 82
Iteration: 3500. Loss: 0.5769420266151428. Accuracy: 83
Iteration: 4000. Loss: 0.44504815340042114. Accuracy: 83
Iteration: 4500. Loss: 0.450328528881073. Accuracy: 83
Iteration: 5000. Loss: 0.2917686700820923. Accuracy: 83
Iteration: 5500. Loss: 0.4468529224395752. Accuracy: 84
Iteration: 6000. Loss: 0.3932793438434601. Accuracy: 84
